<a href="https://colab.research.google.com/github/SERVIR/flood_mapping_intercomparison/blob/main/notebooks/Section_A_Case_Study_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This script will take as an input a csv file containing flood events. In this case, a CSV file was downloaded from the [EM-DAT](https://www.emdat.be/) website. Then, various columns were added to this CSV file. This includes a column called "adm0", which contains the name of the country as it appears in the [FAO GAUL](https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0) dataset. It also added a column called "adm1", that contains the name of the province(s) affected by a particular flood event. Finally, it has a column called "adm2" which contains the name of the district(s) affected by a particular flood event. For adm1 and adm2, the spelling must match the spelling of the districts/contries as they appear in the GAUL dataset. Each flood event will contain a time period of interest and a region of interest. If there is more than one district or province, they must be entered in as a comma-separated list (no spaces).


For each flood event, the script will step through each day listed in the flood event database, and report some properties. For each day, the script will report

* Whether or not there was a Sentinel-1 overpass
* Whether or not there was a Sentinel-2 overpass
* Whether or not there was a Landsat overpass

For all dates where there was a Sentinel-1, Sentinel-2, Landsat-8, or Landsat-9 overpass, the following properties of each sensor (all above sensors + MODIS TERRA, MODIS AQUA, and VIIRS) acquisition will be reported:


* The percent of the area of interest that is covered by the sensor footprint
* The percent of the sensor footprint that is covered by clouds (optical only)
* If there was a Sentinel-1 overpass but no HLS (i.e. either landsat or sentinel-2 overpass), for the sentinel-1 footprint we will report what percentage of pixels are clear-view for both MODIS and VIIRS
* If there was an HLS overpass but no Sentinel-1 overpass, for the HLS overpass, we will report what percentage of pixels are clear-view for VIIRS, MODIS, and HLS
* If there was an HLS overpass as well as a Sentinel-1 overpass, we will
  * Find the overlapping area covered by both Sentinel-1 and HLS sensors
  * For this overlapping area, report the percentage of pixels where HLS, VIIRS, and MODIS all have a clear view of the ground.

From this, we will attempt to find certain "golden goose" flood events that

* Have both a Sentinel-1 and an HLS overpass during the time period of interest
* Have low cloud coverage in HLS, VIIRS, and MODIS sensors
* Ideally occur in SERVIR countries

# Step 1: Import packages & data; Mount Google Drive

In [1]:
import ee
import geemap
import numpy as np
import pandas as pd
from google.colab import drive
import os
drive.mount('/content/drive/')
ee.Authenticate()
ee.Initialize(project = 'servir-sco-assets')

Mounted at /content/drive/


In [2]:
pwd

'/content'

In [17]:
cd drive/My Drive

[Errno 2] No such file or directory: 'drive/My Drive'
/content/drive/My Drive/Flood_Intercomparison/Case_Studies


In [18]:
os.mkdir('Flood_Intercomparison')

In [19]:
cd Flood_Intercomparison

/content/drive/My Drive/Flood_Intercomparison/Case_Studies/Flood_Intercomparison


In [20]:
# Import Raster Data
s1 = ee.ImageCollection("COPERNICUS/S1_GRD")
s2 = ee.ImageCollection("NASA/HLS/HLSS30/v002")                    #("COPERNICUS/S2_SR_HARMONIZED")
l8 = ee.ImageCollection("NASA/HLS/HLSL30/v002")
viirs = ee.ImageCollection("NASA/VIIRS/002/VNP09GA")
terra = ee.ImageCollection("MODIS/061/MOD09GA") # MODIS/061/MOD09GQ
aqua = ee.ImageCollection("MODIS/061/MYD09GA") # MODIS/061/MYD09GQ

# Import Vector Data
gaul = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")

# 2. Define some functions

Now we will write some functions that will help us to do the following for each day of each flood event:

1. viirs_cloud: This function will estimate the cloud cover percentage of the viirs scene
2. modis_cloud: This function will estimate the cloud cover percentage of the modis (AQUA or TERRA) scene
3. hls_scene_and_cloud: This function will determine if there was an HLS overpass (either Sentinel-2 or Landsat 8/9). If there was an overpass, it will estimate the cloud cover percentage of the scene.
4. percent_coverage: This function will determine the proportion of the area covered by a satellite footprint to the area of the region of interest. This function is used for Sentinel-1
5. viirs_and_modis: This function will determine the proportion of the area for which both viirs and modis had clear observations of the ground.
6. all_clouds: This function will determine the proportion of the region of interest for which viirs, modis, and HLS **ALL** had clear observations of the ground
7. date_list_constructor: This function will take the start date and end date of the flood event as defined in our input csv file and create a list of ee.Date objects containing all dates in between the start and end date. This list of dates will be used so that we can run the above six functions for each date of each flood event contained in our input CSV.
8. string_constructor: This function will take a year, month, and day, and create a string in "YYYY-MM-dd" format, which will facilitate the creation of ee.Date objects from the input CSV.
9. region_merger: This function will take a list of regions (defined as featureCollection objects in Earth Engine) and merge them into a single feature collection. This will use the "adm0", "adm1", and "adm2" columns as mentioned in the introduction.

We will use the functions above to create an output CSV. For each flood event we will have a new column for each day that will tell us the outputs of Functions 1-6 above. In the case that there is a Sentinel-1 and a HLS overpass on a certain day, it will tell us what proportion of the region covered by both Sentinel-1 and HLS is free of clouds for viirs, modis, AND HLS. This will help us find our golden goose flood event -- the one with (1) an HLS overpass, (2) a Sentinel-1 overpass, and (3) the highest percentage for which viirs, modis, and HLS all have a clear view of the ground.

Please note that these are **estimates** of cloud cover to give us a general idea of which days of which flood events have good obersvations of the ground. The percentages generated from this script are subject to potential sources of error from the reduceRegion function, where the scale used is roughly equal to the scale of the sensor.

## 2.1: The viirs_cloud Function

In [21]:
def viirs_cloud(img_collection, description, doi, roi):
  whole_area = roi.geometry().area().getInfo()
  rtrn_list = []

  mydate = ee.Date(doi)
  mydate2 = mydate.advance(1, 'day')

  filt = img_collection.filterBounds(roi).filterDate(mydate, mydate2)
  first = filt.first()
  size = filt.size().getInfo()

  if size > 0:
    mos = filt.mosaic().clip(roi)
    single_band = mos.select(first.bandNames().getInfo()[0])
    msk = single_band.mask()

    pix_area = ee.Image().pixelArea()
    pixel_img = pix_area.updateMask(msk)
    proj = mos.projection()

    area_dict = pixel_img.reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = roi.geometry(),
        crs = proj,
        scale = 100,
        maxPixels = 1e13
    )

    area = area_dict.getInfo()["area"]

    ########## Cloud Coverage Percentage ##############
    viirs_qa = mos.select("QF1")                      # Select the Quality Assurance band from the VIIRS image

    viirs_cloud_mask = 1 << 3   # A 1 left-shifted by 3 bits
    viirs_clouds = viirs_qa.bitwiseAnd(viirs_cloud_mask)
    viirs_cloud_area_img = ee.Image.pixelArea().addBands(viirs_clouds)

    viirs_cloud_sum = viirs_cloud_area_img.reduceRegion(**{
        'reducer': ee.Reducer.sum().group(**{
            'groupField': 1,
            'groupName': 'QF1'
        }),
        'geometry': roi,
        'scale': 375,
        'maxPixels': 1e13
    })

    viirs_cloud_dic = viirs_cloud_sum.getInfo()["groups"]
    viirs_cd_len = len(viirs_cloud_dic) # VIIRS cloud dictionary length

    if viirs_cd_len > 1:
      viirs_cloud_pixels = viirs_cloud_dic[1]["sum"]
      viirs_cloud_percentage = viirs_cloud_pixels / area * 100
    else:
      viirs_cloud_percentage = 100                                                                                   # POTENTIAL IMPROVEMENT
    viirs_cloud_string = '{0:0.1f} % of this VIIRS scene is covered by clouds'.format(viirs_cloud_percentage)
    rtrn_list.append(viirs_cloud_string)
  else:
    viirs_percent_string = 'No {} Overpass on this day'.format(description)
    rtrn_list.append(viirs_percent_string)
  return rtrn_list

## 2.2: The modis_cloud Function

In [22]:
def modis_cloud(img_collection, description, doi, roi):
  whole_area = roi.geometry().area().getInfo()
  rtrn_list = []

  mydate = ee.Date(doi)
  mydate2 = mydate.advance(1, 'day')

  filt = img_collection.filterBounds(roi).filterDate(mydate, mydate2)
  first = filt.first()
  size = filt.size().getInfo()

  if size > 0:
    modis_mos = filt.mosaic().clip(roi)
    single_band = modis_mos.select(first.bandNames().getInfo()[0])
    msk = single_band.mask()

    pix_area = ee.Image().pixelArea()
    pixel_img = pix_area.updateMask(msk)
    proj = modis_mos.projection()

    modis_area_dict = pixel_img.reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = roi.geometry(),
        crs = proj,
        scale = 250,
        maxPixels = 1e13
    )

    area = modis_area_dict.getInfo()["area"]

    ################### Cloud Coverage Percentage  ############################
    modis_qa = modis_mos.select("state_1km")

    modis_qa_bit0 = 1 << 0 # A 1 at the 0-bit position
    modis_qa_bit1 = 1 << 1 # A 1 at the 1-bit position

    # If a modis image has a 1 at either bit 0 or bit 1 at the state_1km band, we will count it as a cloud

    modis_cloudy = modis_qa.bitwiseAnd(modis_qa_bit0)
    modis_mixed = modis_qa.bitwiseAnd(modis_qa_bit1)

    modis_cloudy_or_mixed = (modis_cloudy.gt(0)).Or(modis_mixed.gt(0))
    modis_cloud_areaimg = ee.Image.pixelArea().addBands(modis_cloudy_or_mixed) #modis_cloudy_or_mixed

    modis_cloud_sum = modis_cloud_areaimg.reduceRegion(**{
        'reducer': ee.Reducer.sum().group(**{
            'groupField': 1,
            'groupName': 'state_1km'
        }),
        'geometry': roi,
        'scale': 250,
        'maxPixels': 1e13
    })

    modis_cloud_dic = modis_cloud_sum.getInfo()["groups"]
    modis_cd_len = len(modis_cloud_dic)

    if modis_cd_len > 1:
      modis_cloud_pixels = modis_cloud_dic[1]["sum"]
      modis_cloud_percentage = modis_cloud_pixels / area * 100
    else:
      modis_cloud_percentage = 100 # 0

    modis_cloud_string = '{0:0.1f} % of this {1} scene is covered by clouds'.format(modis_cloud_percentage, description)
    rtrn_list.append(modis_cloud_string)
  else:
    percent_string = 'No {} Overpass on this day'.format(description)
    rtrn_list.append(percent_string)
  return rtrn_list

## 2.3: The hls_scene_and_cloud Function

In [23]:
def hls_scene_and_cloud(img_collection, description, doi, roi):
  hls_whole_area = roi.geometry().area().getInfo()
  hls_rtrn_list = []

  mydate = ee.Date(doi)
  mydate2 = mydate.advance(1, 'day')

  filt = img_collection.filterBounds(roi).filterDate(mydate, mydate2)
  first = filt.first()
  size = filt.size().getInfo()

  if size > 0:
    hls_mos = filt.mosaic().clip(roi)
    hls_single_band = hls_mos.select(first.bandNames().getInfo()[0])
    hls_msk = hls_single_band.mask()

    hls_pix_area = ee.Image().pixelArea()
    hls_pixel_img = hls_pix_area.updateMask(hls_msk)
    hls_proj = hls_mos.projection()

    hls_area_dict = hls_pixel_img.reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = roi.geometry(),
        crs = hls_proj,
        scale = 30,
        maxPixels = 1e13
    )

    hls_area = hls_area_dict.getInfo()["area"]
    hls_percent = hls_area / hls_whole_area * 100
    hls_percent_string = 'The {0} scene covers {1:0.1f} % of the region of interest'.format(description, hls_percent)
    hls_rtrn_list.append(hls_percent_string)


    ##################### Cloud Coverage Percentage ###################
    hls_qa = hls_mos.select("Fmask")                                        # Select the Quality Assurance Band

    hls_cloud_bitmask = 1 << 1                      # A 1 left-shifted by 1 bit
    hls_adjacent_bitmask = 1 << 2                   # A 1 left-shifted by 2 bits
    hls_cloud_shadow_bitmask = 1 << 3                # A 1 left-shifted by 3 bits

    hls_clouds = hls_qa.bitwiseAnd(hls_cloud_bitmask)
    hls_adjacent = hls_qa.bitwiseAnd(hls_adjacent_bitmask)
    hls_cloud_shadow = hls_qa.bitwiseAnd(hls_cloud_shadow_bitmask)

    hls_any_clouds = (hls_clouds.gt(0)).Or(hls_clouds.gt(0)).Or(hls_clouds.gt(0))
    hls_cloud_areaimg = ee.Image.pixelArea().addBands(hls_any_clouds)

    hls_cloud_sum = hls_cloud_areaimg.reduceRegion(**{
        'reducer': ee.Reducer.sum().group(**{
            'groupField': 1,
            'groupName': 'Fmask'
        }),
        'geometry': roi,
        'scale': 30,
        'maxPixels': 1e13
    })

    hls_cloud_dic = hls_cloud_sum.getInfo()["groups"]
    hls_cd_len = len(hls_cloud_dic)

    if hls_cd_len > 1:
      hls_cloud_pixels = hls_cloud_dic[1]["sum"]
      hls_cloud_percentage = hls_cloud_pixels / hls_area * 100
    else:
      hls_cloud_percentage = 100

    hls_cloud_string = '{0:0.1f} % of this {1} scene is covered by clouds'.format(hls_cloud_percentage, description)
    hls_rtrn_list.append(hls_cloud_string)
  else:
    percent_string = 'No {} Overpass on this day'.format(description)
    hls_rtrn_list.append(percent_string)

  return hls_rtrn_list

## 2.4: The percent_coverage Function

In [24]:
def percent_coverage(img_collection, description, doi, roi):
  # img_collection is an Earth Engine Image Collection object
  # description is a string that describes the satellite image collection input
  # doi is a string with the date of interest
  # roi is a region of interest represented as an Earth Engine Image Collection Object

  whole_area = roi.geometry().area().getInfo()
  rtrn_list = []

  mydate = ee.Date(doi)
  mydate2 = mydate.advance(1, 'day')

  filt = img_collection.filterBounds(roi).filterDate(mydate, mydate2)
  first = filt.first()
  size = filt.size().getInfo()

  if size > 0:
    mos = filt.mosaic().clip(roi)
    single_band = mos.select(first.bandNames().getInfo()[0])           # Select a random band from the first image
    msk = single_band.mask()

    pix_area = ee.Image().pixelArea()
    pixel_img = pix_area.updateMask(msk)
    proj = mos.projection()

    area_dict = pixel_img.reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = roi.geometry(),
        crs = proj,
        scale = 30,
        maxPixels = 1e13
    )

    area = area_dict.getInfo()["area"]
    percent = area / whole_area * 100
    percent_string = 'The {0} scene covers {1:0.1f} % of the region of interest'.format(description, percent)
    rtrn_list.append(percent_string)
    #rtrn_list.append(area)

  else:
    percent_string = 'No {} Overpass'.format(description)
    rtrn_list.append(percent_string)

  return rtrn_list


## 2.5 The viirs_and_modis function

In [25]:
def viirs_and_modis(this_doi, this_roi):
  ######################## Import datasets  #######################
  this_s1 = ee.ImageCollection("COPERNICUS/S1_GRD")
  this_viirs = ee.ImageCollection("NASA/VIIRS/002/VNP09GA")
  this_terra = ee.ImageCollection("MODIS/061/MOD09GA")
  this_aqua = ee.ImageCollection("MODIS/061/MYD09GA")

  ################### Clip datasets to region and dates of interest  #########
  this_day = ee.Date(this_doi)
  this_day_after = this_day.advance(1, 'day')

  this_s1_roi = this_s1.filterBounds(this_roi).filterDate(this_day, this_day_after)
  this_viirs_roi = this_viirs.filterBounds(this_roi).filterDate(this_day, this_day_after)
  this_terra_roi = this_terra.filterBounds(this_roi).filterDate(this_day, this_day_after)
  this_aqua_roi = this_aqua.filterBounds(this_roi).filterDate(this_day, this_day_after)

  ################### Cloud Masking ############################
  # VIIRS
  this_mos_viirs = this_viirs_roi.mosaic().clip(this_roi)
  this_vqa = this_mos_viirs.select("QF1")
  this_vcm = 1 << 3
  this_vc = this_vqa.bitwiseAnd(this_vcm)
  this_vcai = ee.Image.pixelArea().addBands(this_vc) # VIIRS Cloud area image

  # TERRA
  this_mos_terra = this_terra_roi.mosaic().clip(this_roi)
  this_tqa = this_mos_terra.select("state_1km")

  this_tqa_bit0 =  1 << 0
  this_tqa_bit1 = 1 << 1

  this_t_cloudy = this_tqa.bitwiseAnd(this_tqa_bit0)
  this_t_mixed = this_tqa.bitwiseAnd(this_tqa_bit1)

  this_tcom = (this_t_cloudy.gt(0)).Or(this_t_mixed.gt(0))  # TERRA Cloudy or Mixed
  this_tcai = ee.Image().pixelArea().addBands(this_tcom)

  # AQUA
  this_mos_aqua = this_aqua_roi.mosaic().clip(this_roi)
  this_aqa = this_mos_aqua.select("state_1km")

  this_aqa_bit0 = 1 << 0
  this_aqa_bit1 = 1 << 1

  this_a_cloudy = this_aqa.bitwiseAnd(this_aqa_bit0)
  this_a_mixed = this_aqa.bitwiseAnd(this_aqa_bit1)

  this_acom = (this_a_cloudy.gt(0)).Or(this_a_mixed.gt(0))      # AQUA Cloudy or Mixed
  this_acai = ee.Image().pixelArea().addBands(this_acom)   # Aqua Cloud Area Image

  ################### Combining #################################
  this_aqua_msk = this_acom.mask()
  this_terra_msk = this_tcom.mask()

  this_a_mod = this_acai.unmask().where(this_aqua_msk.eq(0), ee.Image(2)).clip(this_roi)
  this_t_mod = this_tcai.unmask().where(this_terra_msk.eq(0), ee.Image(2)).clip(this_roi)

  this_modis = this_a_mod.eq(0).Or(this_t_mod.eq(0)).select("state_1km")
  this_final_viirs = this_vcai.select("QF1")

  this_clear = this_modis.eq(1).And(this_final_viirs.neq(0))

  #################### Calculating area #################################
  this_cloudfinal = ee.Image().pixelArea().addBands(this_clear)

  this_cloudsum = this_cloudfinal.reduceRegion(**{
      'reducer': ee.Reducer.sum().group(**{
          'groupField': 1,
          'groupName': "state_1km"
      }),
      'geometry': this_roi,
      'scale': 100,
      'maxPixels': 1e13
  })

  this_cloud_dict = this_cloudsum.getInfo()["groups"]

  if len(this_cloud_dict) > 1:
    this_unclear_pix = this_cloud_dict[0]["sum"]
    this_clear_pix = this_cloud_dict[1]["sum"]
    this_tot_pix = this_unclear_pix + this_clear_pix
    this_percent = this_clear_pix / this_tot_pix * 100
  else:
    if len(this_cloud_dict) == 0:
      this_percent = 0.1
    else:
      this_cloud_or_not = this_cloud_dict[0]["state_1km"]
      if this_cloud_or_not == 0:
        this_percent = 0.2
      else:
        this_percent = 99.8
        this_cd_length = len(this_cloudsum.getInfo())

  #if this_cd_length > 1:                                              # Avoids the edge case where the image is 100% clouds
    #this_clear_pix = this_cloudsum.getInfo()["groups"][1]["sum"]
    #this_tot_area = this_roi.area().getInfo()
    #this_percent = this_clear_pix / this_tot_area * 100
  #else:
    #this_percent = 0

  viirs_and_mod_string = '{0:0.1f} % of the region is clear for both viirs and modis'.format(this_percent)

  return viirs_and_mod_string

# 2.6 The all_clouds_v2 function

In [26]:
'''
def all_clouds_v2(the_doi, the_roi):

  # Import Datasets
  the_s1 = ee.ImageCollection("COPERNICUS/S1_GRD")
  the_s2 = ee.ImageCollection("NASA/HLS/HLSS30/v002")
  the_l8 = ee.ImageCollection("NASA/HLS/HLSL30/v002")
  the_viirs = ee.ImageCollection("NASA/VIIRS/002/VNP09GA")
  the_terra = ee.ImageCollection("MODIS/061/MOD09GA")
  the_aqua = ee.ImageCollection("MODIS/061/MYD09GA")

  # Clip datasets to the region and date of interest
  the_day = ee.Date(the_doi)
  the_day_after = the_day.advance(1, 'day')

  s1_roi = the_s1.filterBounds(the_roi).filterDate(the_day, the_day_after)
  s2_roi = the_s2.filterBounds(the_roi).filterDate(the_day, the_day_after)
  l8_roi = the_l8.filterBounds(the_roi).filterDate(the_day, the_day_after)
  viirs_roi = the_viirs.filterBounds(the_roi).filterDate(the_day, the_day_after)
  terra_roi = the_terra.filterBounds(the_roi).filterDate(the_day, the_day_after)
  aqua_roi = the_aqua.filterBounds(the_roi).filterDate(the_day, the_day_after)

  ##################### Cloud Masking   ####################################

  # VIIRS
  mos_viirs = viirs_roi.mosaic().clip(the_roi)
  vqa = mos_viirs.select("QF1")
  vcm = 1 <<3
  vc = vqa.bitwiseAnd(vcm)
  vcai = ee.Image.pixelArea().addBands(vc) # VIIRS Cloud area image

  # TERRA
  mos_terra = terra_roi.mosaic().clip(the_roi)
  tqa = mos_terra.select("state_1km")

  tqa_bit0 =  1 << 0
  tqa_bit1 = 1 << 1

  t_cloudy = tqa.bitwiseAnd(tqa_bit0)
  t_mixed = tqa.bitwiseAnd(tqa_bit1)

  tcom = (t_cloudy.gt(0)).Or(t_mixed.gt(0))  # TERRA Cloudy or Mixed
  tcai = ee.Image().pixelArea().addBands(tcom)

  # AQUA
  mos_aqua = aqua_roi.mosaic().clip(the_roi)
  aqa = mos_aqua.select("state_1km")

  aqa_bit0 = 1 << 0
  aqa_bit1 = 1 << 1

  a_cloudy = aqa.bitwiseAnd(aqa_bit0)
  a_mixed = aqa.bitwiseAnd(aqa_bit1)

  acom = (a_cloudy.gt(0)).Or(a_mixed.gt(0))      # AQUA Cloudy or Mixed
  acai = ee.Image().pixelArea().addBands(acom)   # Aqua Cloud Area Image

  # Landsat
  mos_landsat = l8_roi.mosaic().clip(the_roi)
  mos_sentinel = s2_roi.mosaic().clip(the_roi)

  num_landsat = len(mos_landsat.bandNames().getInfo())
  num_sentinel = len(mos_sentinel.bandNames().getInfo())

  if num_landsat > 0 or num_sentinel > 0:            # and
    lqa = mos_landsat.select("Fmask")
    sqa = mos_sentinel.select("Fmask")

    lcm = 1 << 3
    lc = lqa.bitwiseAnd(lcm)
    lcai = ee.Image().pixelArea().addBands(lc)

    scm = 1 << 3
    sc = sqa.bitwiseAnd(scm)
    scai = ee.Image().pixelArea().addBands(sc)

    #############################################################

    ################ Combining ##################################
    aqua_msk = acom.mask()
    terra_msk = tcom.mask()
    sentinel_msk = sc.mask()
    landsat_msk = lc.mask()

    a_mod = acai.unmask().where(aqua_msk.eq(0), ee.Image(2)).clip(the_roi)
    t_mod = tcai.unmask().where(terra_msk.eq(0), ee.Image(2)).clip(the_roi)
    s_mod = scai.unmask().where(sentinel_msk.eq(0), ee.Image(2)).clip(the_roi)
    l_mod = lcai.unmask().where(landsat_msk.eq(0), ee.Image(2)).clip(the_roi)

    the_hls = l_mod.eq(0).Or(s_mod.eq(0)).select("Fmask")
    the_modis = a_mod.eq(0).Or(t_mod.eq(0)).select("state_1km")
    final_viirs = vcai.select("QF1")

    clear_view = the_hls.eq(1).And(the_modis.eq(1)).And(final_viirs.neq(0))

    ################ Calculating Area #############################
    cloudfinal_area_img = ee.Image().pixelArea().addBands(clear_view)

    cloudfinal_sum = cloudfinal_area_img.reduceRegion(**{
        'reducer': ee.Reducer.sum().group(**{
            'groupField': 1,
            'groupName': 'Fmask'
        }),
        'geometry': the_roi,#.geometry(),
        'scale': 100,
        'maxPixels': 1e13
    })

    cloud_dict = cloudfinal_sum.getInfo()["groups"]

    if len(cloud_dict) > 1:
      pixels_unclear = cloud_dict[0]["sum"]
      pixels_clear = cloudfinal_sum.getInfo()["groups"][1]["sum"]
      pixels_total = pixels_unclear + pixels_clear
      #total_area = the_roi.area().getInfo()

      percent = pixels_clear / pixels_total * 100

    else:
      if len(cloud_dict) == 0:
        percent = 0.1
      else:
        cloud_or_not = cloud_dict[0]["Fmask"]
        if cloud_or_not == 0:
          percent = 0.2
        else:
          percent = 99.7
  else:
    percent = 0.1

  allcloudstring = '{0:0.2f} % of the region occupied by both Sentinel-1 and HLS is clear for all sensors'.format(percent)

  return allcloudstring
'''

In [39]:
def all_clouds(the_doi, the_roi):

  # Import Datasets
  the_s1 = ee.ImageCollection("COPERNICUS/S1_GRD")
  the_s2 = ee.ImageCollection("NASA/HLS/HLSS30/v002")
  the_l8 = ee.ImageCollection("NASA/HLS/HLSL30/v002")
  the_viirs = ee.ImageCollection("NASA/VIIRS/002/VNP09GA")
  the_terra = ee.ImageCollection("MODIS/061/MOD09GA")
  the_aqua = ee.ImageCollection("MODIS/061/MYD09GA")

  # Clip datasets to the region and date of interest
  the_day = ee.Date(the_doi)
  the_day_after = the_day.advance(1, 'day')

  s1_roi = the_s1.filterBounds(the_roi).filterDate(the_day, the_day_after)
  s2_roi = the_s2.filterBounds(the_roi).filterDate(the_day, the_day_after)
  l8_roi = the_l8.filterBounds(the_roi).filterDate(the_day, the_day_after)
  viirs_roi = the_viirs.filterBounds(the_roi).filterDate(the_day, the_day_after)
  terra_roi = the_terra.filterBounds(the_roi).filterDate(the_day, the_day_after)
  aqua_roi = the_aqua.filterBounds(the_roi).filterDate(the_day, the_day_after)

  ##################### Cloud Masking   ####################################

  # VIIRS
  mos_viirs = viirs_roi.mosaic().clip(the_roi)
  vqa = mos_viirs.select("QF1")
  vcm = 1 <<3
  vc = vqa.bitwiseAnd(vcm)
  vcai = ee.Image.pixelArea().addBands(vc) # VIIRS Cloud area image

  # TERRA
  mos_terra = terra_roi.mosaic().clip(the_roi)
  tqa = mos_terra.select("state_1km")

  tqa_bit0 =  1 << 0
  tqa_bit1 = 1 << 1

  t_cloudy = tqa.bitwiseAnd(tqa_bit0)
  t_mixed = tqa.bitwiseAnd(tqa_bit1)

  tcom = (t_cloudy.gt(0)).Or(t_mixed.gt(0))  # TERRA Cloudy or Mixed
  tcai = ee.Image().pixelArea().addBands(tcom)

  # AQUA
  mos_aqua = aqua_roi.mosaic().clip(the_roi)
  aqa = mos_aqua.select("state_1km")

  aqa_bit0 = 1 << 0
  aqa_bit1 = 1 << 1

  a_cloudy = aqa.bitwiseAnd(aqa_bit0)
  a_mixed = aqa.bitwiseAnd(aqa_bit1)

  acom = (a_cloudy.gt(0)).Or(a_mixed.gt(0))      # AQUA Cloudy or Mixed
  acai = ee.Image().pixelArea().addBands(acom)   # Aqua Cloud Area Image

  # Landsat
  mos_landsat = l8_roi.mosaic().clip(the_roi)
  mos_sentinel = s2_roi.mosaic().clip(the_roi)

  num_landsat = len(mos_landsat.bandNames().getInfo())
  num_sentinel = len(mos_sentinel.bandNames().getInfo())

  if num_landsat > 0 or num_sentinel > 0:            # and
    if num_landsat > 0:
      lqa = mos_landsat.select("Fmask")
      sqa = ee.Image().rename("Fmask")
    if num_sentinel > 0:
      sqa = mos_sentinel.select("Fmask")
      lqa = ee.Image().rename("Fmask")

    lcm = 1 << 3
    lc = lqa.bitwiseAnd(lcm)
    lcai = ee.Image().pixelArea().addBands(lc)

    scm = 1 << 3
    sc = sqa.bitwiseAnd(scm)
    scai = ee.Image().pixelArea().addBands(sc)

    #############################################################

    ################ Combining ##################################
    aqua_msk = acom.mask()
    terra_msk = tcom.mask()
    sentinel_msk = sc.mask()
    landsat_msk = lc.mask()

    a_mod = acai.unmask().where(aqua_msk.eq(0), ee.Image(2)).clip(the_roi)
    t_mod = tcai.unmask().where(terra_msk.eq(0), ee.Image(2)).clip(the_roi)
    s_mod = scai.unmask().where(sentinel_msk.eq(0), ee.Image(2)).clip(the_roi)
    l_mod = lcai.unmask().where(landsat_msk.eq(0), ee.Image(2)).clip(the_roi)

    the_hls = l_mod.eq(0).Or(s_mod.eq(0)).select("Fmask")
    the_modis = a_mod.eq(0).Or(t_mod.eq(0)).select("state_1km")
    final_viirs = vcai.select("QF1")

    clear_view = the_hls.eq(1).And(the_modis.eq(1)).And(final_viirs.neq(0))

    ################ Calculating Area #############################
    cloudfinal_area_img = ee.Image().pixelArea().addBands(clear_view)

    cloudfinal_sum = cloudfinal_area_img.reduceRegion(**{
        'reducer': ee.Reducer.sum().group(**{
            'groupField': 1,
            'groupName': 'Fmask'
        }),
        'geometry': the_roi,#.geometry(),
        'scale': 30,
        'maxPixels': 1e13
    })

    cloud_dict = cloudfinal_sum.getInfo()["groups"]

    if len(cloud_dict) > 1:
      pixels_unclear = cloud_dict[0]["sum"]
      pixels_clear = cloudfinal_sum.getInfo()["groups"][1]["sum"]
      pixels_total = pixels_unclear + pixels_clear
      #total_area = the_roi.area().getInfo()

      percent = pixels_clear / pixels_total * 100

    else:
      if len(cloud_dict) == 0:
        percent = 0.1
      else:
        cloud_or_not = cloud_dict[0]["Fmask"]
        if cloud_or_not == 0:
          percent = 0.2
        else:
          percent = 99.7
  else:
    percent = 0.1

  allcloudstring = '{0:0.2f} % of the region occupied by both Sentinel-1 and HLS is clear for all sensors'.format(percent)

  return allcloudstring

# 2.7: The date_list_constructor function

In [28]:
def date_list_constructor(start_date, end_date):
  list_of_dates = []
  start_ee = ee.Date(start_date)
  end_ee = ee.Date(end_date)

  iterator = start_ee

  w = end_ee.difference(iterator, 'day').getInfo()
  while w != -1:
    list_of_dates.append(iterator)
    iterator = iterator.advance(1, 'day')
    w = end_ee.difference(iterator, 'day').getInfo()

  return list_of_dates

# 2.8: The string_constructor function

In [29]:
def string_constructor(year_, month_, day_):
  year_string = str(year_)

  if month_ < 10:
    month_string = '0' + str(month_)
  else:
    month_string = str(month_)

  if day_ < 10:
    day_string = '0' + str(int(day_))
  else:
    day_string = str(int(day_))

  rtrn_str = year_string + '-' + month_string + '-' + day_string
  return rtrn_str

# 2.9 The region_merger function

In [30]:
def region_merger(input_string, column_name):
  area_of_interest = ee.FeatureCollection([])

  splitstring = input_string.split(",")
  feature_list = []
  for v in splitstring:
    feature = gaul.filter(ee.Filter.eq(column_name, v))
    feature_list.append(feature)
  for x in feature_list:
    area_of_interest = area_of_interest.merge(x)

  return area_of_interest

# Chapter 3: Creating our CSV

Navigate to the Flood_Intercomparison/Case_Studies folder in your google drive. write in the filename here containing the path to your CSV. CSV's must have the following formatting:



*   Each row contains a case study event with the following columns
*   adm0: Contains the spelling of the country as it appears in the GAUL dataset.
* adm1: Contains the spelling of the province(s) (level 1 administrative boundary) as it appears in the GAUL dataset.
  * If more than one adm2, must be entered as a comma separated list with no spaces between
* adm2: Contains the spelling of the district(s) (level 2 administrative boundary) as it appears in the GAUL dataset.
  * If more than one adm2, must be entered as a comma separated list with no spaces between
* Start Year: Contains the year in which the flood event started
* Start Month: Contains the month in which the flood event started
* Start Day: Contains the day in which the flood event started
* End Year: Contains the year in which the flood event ended.
* End Month: Contains the month in which the flood event ended.
* End Day: Contains the day in which the flood event ended.

An example CSV can be found by [clicking here](https://docs.google.com/spreadsheets/d/1rmKgp6CxUQ9gdVNeHjmXECby-4SDku5vA4GHPkLR-no/edit?usp=sharing)


In [15]:
filename = 'perifinal_01_15_25_modded_emdat_global_2024_post_august.csv'

flood_events = pd.read_csv(filename, encoding = 'latin-1')

flood_events.head()

,DisNo.,Historic,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,External IDs,Event Name,ISO,...,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damage ('000 US$),"Insured Damage, Adjusted ('000 US$)",Total Damage ('000 US$),"Total Damage, Adjusted ('000 US$)",CPI,Admin Units,Entry Date,Last Update
0,2024-0561-IND,No,nat-hyd-flo-flo,Natural,Hydrological,Flood,Flood (General),NaN,NaN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/1/2024,8/23/2024
1,2024-0569-TCD,No,nat-hyd-flo-flo,Natural,Hydrological,Flood,Flood (General),GLIDE:FL-2024-000139,NaN,TCD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/5/2024,10/22/2024
2,2024-0580-YEM,No,nat-hyd-flo-flo,Natural,Hydrological,Flood,Flood (General),NaN,NaN,YEM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/8/2024,8/19/2024
3,2024-0617-GIN,No,nat-hyd-flo-flo,Natural,Hydrological,Flood,Flood (General),GLIDE:FL-2024-000148,NaN,GIN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/21/2024,11/12/2024
4,2024-0624-BGD,No,nat-hyd-flo-fla,Natural,Hydrological,Flood,Flash flood,NaN,NaN,BGD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/22/2024,10/22/2024


Since this is a memory intensive code, if you have a CSV with a lot of rows, you may want to run the code row by row. If this is the case, change the portion of the code that says "if c == _" to be equivalent to the row number (using 0 for the first row). If you want to run the code for the entire csv, you can just change it to c < 10000. The code will also print some of the outputs so you can monitor the progress of the code. For flood events containing a large area of interest or a long range of dates, the below code cell can take hours to run for a single flood event.

** IMPORTANT: Each time you run the code, change the "my_file" variable to a different file name. If you don't, the previous outputs from the code will be overwritten.**

In [ ]:
import csv
#my_output_file = 'full_test_hytgh.csv'
my_file = 'IOTA_flood_events_pt16.csv'

fields = ['Start Date', 'End Date', 'Country', 'Gaul Regions']

rows = []

num_rows = len(flood_events)


for c in range(num_rows):
  if c == 16:
    #completion_percentage = c/num_rows * 100
    #print("{0:0.1f} % Complete!".format(completion_percentage))
    #print(c)

    row_of_interest = flood_events.iloc[c]          # Iterate through rows of spreadsheet
    my_country = row_of_interest.loc['adm0']          # Extract the location and time of the flood event
    my_provinces = row_of_interest.loc['adm1']
    my_districts = row_of_interest.loc['adm2']
    my_start_year = row_of_interest.loc['Start Year']
    my_start_month = row_of_interest.loc['Start Month']
    my_start_day = row_of_interest.loc['Start Day']
    my_end_year = row_of_interest.loc['End Year']
    my_end_month = row_of_interest.loc['End Month']
    my_end_day = row_of_interest.loc['End Day']

    # Build the Start Date and End Date Strings
    my_start_date = string_constructor(my_start_year, my_start_month, my_start_day)
    my_end_date = string_constructor(my_end_year, my_end_month, my_end_day)

    # Create the region of interest feature collection
    if type(my_provinces) == str:
      roistring = my_provinces
      my_aoi = region_merger(roistring, 'ADM1_NAME')
    else:
      roistring = my_districts
      my_aoi = region_merger(roistring, 'ADM2_NAME')

    # Construct the List of Dates between start date and end_date
    my_date_list = date_list_constructor(my_start_date, my_end_date)

    # For each date in my_date_list, report the cloud and overpass information
    cloud_info = []               # Create an empty list
    for d in range(len(my_date_list)):         # For each date of the flood event, do the following:
      nest = []                                # Create an empty list
      iterated = my_date_list[d]               # Get the date of interest

      get_year = str(iterated.get('year').getInfo())
      get_month = str(iterated.get('month').getInfo())
      get_day = str(iterated.get('day').getInfo())
      get_date = get_year + '/' + get_month + '/' + get_day
      print(get_date)

      day_after = iterated.advance(1, 'day')

      s1_filt = s1.filterBounds(my_aoi).filterDate(iterated, day_after)
      viirs_filt = viirs.filterBounds(my_aoi).filterDate(iterated, day_after)
      s2_filt = s2.filterBounds(my_aoi).filterDate(iterated, day_after)
      l8_filt = l8.filterBounds(my_aoi).filterDate(iterated, day_after)
      terra_filt = terra.filterBounds(my_aoi).filterDate(iterated, day_after)
      aqua_filt = aqua.filterBounds(my_aoi).filterDate(iterated, day_after)

      s1_info = percent_coverage(s1_filt, 'Sentinel-1', iterated, my_aoi)
      #viirs_info = viirs_cloud(viirs_filt, 'VIIRS', iterated, my_aoi)
      s2_info = hls_scene_and_cloud(s2_filt, 'Sentinel-2', iterated, my_aoi)
      l8_info = hls_scene_and_cloud(l8_filt, 'Landsat', iterated, my_aoi)
      print(s1_info)
      print(s2_info)
      print(l8_info)

      nest.append(get_date)
      nest.append(s1_info)
      nest.append(s2_info)
      nest.append(l8_info)

      if s1_info[0][0] == 'T':
        if s2_info[0][0] == 'T' or l8_info[0][0] == 'T':
          print("synchronous HLS and S1 overpass")

          viirs_info = viirs_cloud(viirs_filt, 'VIIRS', iterated, my_aoi)
          terra_info = modis_cloud(terra_filt, 'TERRA', iterated, my_aoi)
          aqua_info = modis_cloud(aqua_filt, 'AQUA', iterated, my_aoi)

          nest.append(viirs_info)
          nest.append(terra_info)
          nest.append(aqua_info)

          s1_footprint = ee.FeatureCollection(s1_filt.geometry()).union()
          s2_footprint = ee.FeatureCollection(s2_filt.geometry()).union()
          l8_footprint = ee.FeatureCollection(l8_filt.geometry()).union()

          s1_feat = ee.Feature(s1_footprint.first())
          s2_feat = ee.Feature(s2_footprint.first())
          l8_feat = ee.Feature(l8_footprint.first())

          hls_footprint = s2_feat.union(l8_feat)
          aoi_geom = my_aoi.geometry()
          hls_and_s1 = hls_footprint.intersection(s1_feat).intersection(aoi_geom)

          s1hls_area = hls_and_s1.geometry().area().getInfo()
          roi_area =  my_aoi.geometry().area().getInfo()

          proportion = s1hls_area / roi_area * 100
          sync_string = 'The area covered by both HLS and S1 covers {0:0.1f} % of the area of interest'.format(proportion)
          print(sync_string)
          nest.append(sync_string)

          if proportion > 1:
            all_test = all_clouds(iterated, aoi_geom)  # all_clouds_v2         #hls_and_s1.geometry()
            print(all_test)
            nest.append(all_test)
          else:
            no_overlap_string = "S1 and HLS footprints do NOT overlap"
            print(no_overlap_string)
            nest.append(no_overlap_string)

          #s2_sync = hls_scene_and_cloud(s2_filt, 'Sentinel-2 synchronous area', iterated, hls_and_s1)
          #all_test = all_clouds(iterated, hls_and_s1.geometry())
          #print(all_test)
          #nest.append(all_test)
          #s2_sync_info = hls_cloud(s2_filt, 'sentinel-2 sync', iterated, hls_and_s1.geometry())
          #terra_sync_info = modis_cloud_sync(terra_filt)

          #print(sync_string)
          #print(s2_sync_info[0])
          #nest.append(sync_string)
          #nest.append(s2_sync_info[0])
        else:
          print('s1 overpass but no HLS overpass')
          this_s1_footprint = ee.FeatureCollection(s1_filt.geometry()).union()
          this_s1_feat = ee.Feature(this_s1_footprint.first())
          this_aoi_geom = my_aoi.geometry()
          s1_final = this_s1_feat.intersection(this_aoi_geom)

          s1_area = this_s1_feat.geometry().area().getInfo()
          this_roi_area = my_aoi.geometry().area().getInfo()
          this_proportion = s1_area / this_roi_area * 100

          vm_test = viirs_and_modis(iterated, this_s1_feat.geometry())
          viirs_info = viirs_cloud(viirs_filt, 'VIIRS', iterated, my_aoi)
          terra_info = modis_cloud(terra_filt, 'TERRA', iterated, my_aoi)
          aqua_info = modis_cloud(aqua_filt, 'AQUA', iterated, my_aoi)
          print(vm_test)
          nest.append(vm_test)
          nest.append(viirs_info)
          nest.append(terra_info)
          nest.append(aqua_info)

      else:
        print('no s1 overpass')
        vm_info = viirs_and_modis(iterated, my_aoi.geometry())
        viirs_info = viirs_cloud(viirs_filt, 'VIIRS', iterated, my_aoi)
        terra_info = modis_cloud(terra_filt, 'TERRA', iterated, my_aoi)
        aqua_info = modis_cloud(aqua_filt, 'AQUA', iterated, my_aoi)

        nest.append(vm_info)
        nest.append(viirs_info)
        nest.append(terra_info)
        nest.append(aqua_info)
        print(vm_info)
        #nest.append(terra_info)


      #print(s1_info)
      #print(s2_info)
      #print(l8_info)

      #nest.append(get_date)
      #nest.append(s1_info)
      #nest.append(viirs_info)
      #nest.append(s2_info)
      #nest.append(l8_info)
      #nest.append(terra_info)
      #nest.append(aqua_info)
      #nest.append(viirs_info)

      cloud_info.append(nest)

  # Add the information to the Output List
    output_row = [my_start_date, my_end_date, my_country, roistring] #,cloud_info

    for e in range(len(cloud_info)):
      output_row.append(cloud_info[e])
    rows.append(output_row)

# Write to CSV file


with open(my_file, 'w') as csvfile:
  csvwriter = csv.writer(csvfile)
  csvwriter.writerow(fields)
  csvwriter.writerows(rows)

2024/8/1
['The Sentinel-1 scene covers 0.2 % of the region of interest']
['The Sentinel-2 scene covers 10.6 % of the region of interest', '97.9 % of this Sentinel-2 scene is covered by clouds']
['No Landsat Overpass on this day']
synchronous HLS and S1 overpass
The area covered by both HLS and S1 covers 0.0 % of the area of interest
S1 and HLS footprints do NOT overlap
2024/8/2
['No Sentinel-1 Overpass']
['The Sentinel-2 scene covers 23.0 % of the region of interest', '45.2 % of this Sentinel-2 scene is covered by clouds']
['The Landsat scene covers 9.4 % of the region of interest', '54.0 % of this Landsat scene is covered by clouds']
no s1 overpass
13.4 % of the region is clear for both viirs and modis
2024/8/3
['The Sentinel-1 scene covers 33.0 % of the region of interest']
['The Sentinel-2 scene covers 2.1 % of the region of interest', '13.0 % of this Sentinel-2 scene is covered by clouds']
['The Landsat scene covers 17.4 % of the region of interest', '0.7 % of this Landsat scene is